# Primena GCN-a nad Cora skupom podataka

Cilj je predvideti vrstu naučnog rada na osnovu citiranih radova\
Skup podataka je dat kao niz čvorova koji predstavljaju atribute radova, dok grana između čvora $i$ i čvora $j$ označava da rad $i$ citira rad $j$ 

In [3]:
%pip install torch_geometric torch

   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/203.0 MB 4.6 MB/s eta 0:00:45
   ---------------------------------------- 1.7/203.0 MB 18.4 MB/s eta 0:00:11
   - -------------------------------------- 6.2/203.0 MB 43.9 MB/s eta 0:00:05
   -- ------------------------------------- 10.9/203.0 MB 81.8 MB/s eta 0:00:03
   -- ------------------------------------- 14.6/203.0 MB 93.9 MB/s eta 0:00:03
   ---- ---------------------------------- 21.2/203.0 MB 108.8 MB/s eta 0:00:02
   ----- --------------------------------- 27.5/203.0 MB 131.2 MB/s eta 0:00:02
   ------ -------------------------------- 32.9/203.0 MB 131.2 MB/s eta 0:00:02
   ------- ------------------------------- 38.4/203.0 MB 108.8 MB/s eta 0:00:02
   -------- ------------------------------ 43.8/203.0 MB 108.8 MB/s eta 0:00:02
   --------- ------------------------------ 47.6/203.0 MB 93.0 MB/s eta 0:00:02
   ---------- ---------------------------- 54.4/203.0


[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
import pandas as pd
import models

In [5]:
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())


Processing...
Done!


In [7]:
print(f'Broj grafova: {len(dataset)}')
print(f'Feature-a po cvoru: {dataset.num_features}')
print(f'Broj klasa: {dataset.num_classes}')
print(50*'=')

data = dataset[0]  

print(data)
print(f'Broj cvorova: {data.num_nodes}')
print(f'Broj grana: {data.num_edges}')
print(f'Broj uzoraka za treniranje: {data.train_mask.sum()}')
print(f'Graf nije orijentisan: {data.is_undirected()}')

Broj grafova: 1
Feature-a po cvoru: 1433
Broj klasa: 7
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Broj cvorova: 2708
Broj grana: 10556
Broj uzoraka za treniranje: 140
Graf nije orijentisan: True


Za trening potrebne su nam 3 matrice
- A - matrica susedstva, velicine 2708x2708
- X - matrica feature-a, velicine 2708x1433
- y - niz ciljnih atributa - u ovom slucaju samo 140 cvorova imaju labele

In [14]:
X = data.x
y = data.y
train_mask = data.train_mask
test_mask = data.test_mask

In [15]:
data.edge_index

tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])

In [9]:
result_df = pd.DataFrame()

In [ ]:
input_dim = data.num_features
output_dim = data.num_classes
hidden_dims = [[8],[8,8],[8,8,8],[8,8,8,8],[16],[16,16],[16,16,16]]

for hd in hidden_dims:
    gcn = models.GCN(input_dim, hd, output_dim)
    gcn.train()